In [1]:
import time
import os
import json
from datetime import datetime
import math

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
from tqdm import tqdm_notebook

In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.common import exceptions
from collections import defaultdict
from copy import deepcopy
from tqdm import tqdm_notebook

http://consumer.fss.or.kr/fss/consumer/flguide/beware/faq/list.jsp?url=/fss/cm/1208250172921&bbsid=1208250172921&page=%s

1~94page까지

In [21]:
def get_qnaList(url):

    req = requests.get(url, timeout = 30)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')

    qna_list = soup.select('td.tit > a')

    for idx, qna in enumerate(qna_list):
        qna_list[idx] = 'http://consumer.fss.or.kr/fss/consumer/flguide/beware/faq/' + qna['href']

    return qna_list

In [22]:
def get_qnaContent(url):
    req = requests.get(url)
    cont = req.content
    soup = BeautifulSoup(cont, 'html.parser')
    
    qna = dict()
    qna['제목'] = soup.select_one('thead > tr > td').text
    qna['작성자'] = soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(1) > td:nth-of-type(1)').text 
    qna['등록일'] =soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(1) > td:nth-of-type(2)').text
    qna['조회수'] =soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(1) > td:nth-of-type(3)').text
    qna['담당부서'] = soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(2) > td:nth-of-type(1)').text
    qna['담당팀'] = soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(2) > td:nth-of-type(2)').text
    qna['문의'] = soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(2) > td:nth-of-type(3)').text
    
    try:
        qna['첨부파일'] = soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(3) > td > a')['href']
    except:
        try:
            qna['첨부파일'] = soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(3) > td > a').text
        except:
            qna['첨부파일'] = 'NULL'
    qna['답변'] = soup.select_one('#con_box > table:nth-of-type(1) > tbody > tr:nth-of-type(4) > td').text
  
    return qna

## 아래의 메인함수 -> 서버 끊김 문제 발생

In [23]:
def main():
    total_qnaList = list()
    fileName = '5_금융감독원(서민금융1332)'
    csv_file = open(str(fileName) + '.csv', "w", encoding = 'utf-8-sig', newline = '')
    
    columns = ['제목', '작성자', '등록일', '조회수', '담당부서', '담당팀', '문의', '첨부파일','답변']
    
    wr = csv.writer(csv_file)    
    wr.writerow(columns)
    
    for idx in tqdm_notebook(range(1, 94)):
        url = 'http://consumer.fss.or.kr/fss/consumer/flguide/beware/faq/list.jsp?url=/fss/cm/1208250172921&bbsid=1208250172921&page=%s'%(str(idx))
        qnaList = get_qnaList(url)
        
        for qna in qnaList:
            qnaContent = get_qnaContent(qna)
            print(qnaContent)
            total_qnaList.append(qnaContent)
        print(idx, '페이지 완료')
    
    
    # '모든 수집 복지서비스'에 대한 반복문 -> 복지 서비스 하나
    for info_dict in total_qnaList:
        # 'columns'을 돌면서 info_dict가 해당 col을 가지고 있으면 정보를 적고, 없으면 NULL
        wr.writerow([info_dict.get(col, 'NULL') for col in columns])
    
    print('==================== 저장 완료 =======================')
    csv_file.close()
             
    

In [24]:
if __name__ == "__main__":
    main()

{'제목': '소비자보호실태평가 결과는 어디서 확인할 수 있나요?', '작성자': '전효희', '등록일': '2018-04-27 05:45:41', '조회수': '14149', '담당부서': '금융소비자보호총괄국', '담당팀': '소비자보호제도팀', '문의': '02-3145-5694', '첨부파일': 'NULL', '답변': '\n\xa0소비자보호실태평가 결과는 금융감독원 홈페이지(www.fss.or.kr) 또는 개별 금융기관 및 협회 홈페이지* 등을 통해 확인하실 수 있습니다.\n\xa0 \n※ 금융 권역별 협회 홈페이지 목록\n\xa0 \n은\xa0\xa0\xa0\xa0\xa0 행 : 전국은행연합회(www.kfb.or.kr)\n생명보험 : 생명보험협회(www.klia.or.kr)\n손해보험 : 손해보험협회(www.knia.or.kr)\n카\xa0\xa0\xa0\xa0\xa0\xa0드 : 여신금융협회(www.crefia.or.kr)\n저축은행 : 저축은행중앙회(www.fsb.or.kr)\n금융투자 : 금융투자협회(www.kofia.or.kr) \n'}
{'제목': '소비자보호실태평가의 평가방법이 궁금합니다.', '작성자': '전효희', '등록일': '2018-04-27 05:44:23', '조회수': '11802', '담당부서': '금융소비자보호총괄국', '담당팀': '소비자보호제도팀', '문의': '02-3145-5694', '첨부파일': 'http://www.fss.or.kr/download.bbs?bbsid=1208250172921&fidx=1565325572698', '답변': '\n\xa0\n「금융소비자보호 모범규준」에 명기된 소비자보호 사항의 이행여부 점검을 위해 계량 5개, 비계량 5개 등 총 10개 부문으로 구성하여 각 부문별로 평가하고 있습니다. \n\xa0 \n자세한 내용은 첨부파일을 참고해주시길 바랍니다.'}
{'제목': '소비자보호실태평가는 어떤 제도인가요?', '작성자': '전효희', '등록일': '2018-04-27 05:37:53

{'제목': '은행에 대출을 신청했으나, 거절되었는데 그 이유를 정확히 모르겠습니다.', '작성자': '안상현', '등록일': '2014-09-22 10:47:28', '조회수': '21650', '담당부서': '금융민원실', '담당팀': '은행민원팀', '문의': '3145-5772', '첨부파일': 'NULL', '답변': '\r\n\r\n\r\n\r\n                대출이란 금융회사가 자율적으로 차주의 신용상태를 고려하여 향후 회수가능성 등을 심사한 후 자금을 빌려주는 것입니다.\n\r\n또한, 통상 은행의 대출금 회수가능성에 대한 심사시 기존 대출에 대한 연체 여부, 직장 유무, 소득규모 등 대출신청자의 신용상태 및 담보제공의 경우 담보 가치 등을 감안하기 때문에 이에 미흡할 경우에는 대출신청이 거절될 수 있는 것입니다.\n\r\n다만, 대출이 거절된 경우 금융소비자들은 그 사유에 대해 궁금할텐데요. 금융감독원은 최근 은행으로 하여금 대출신청자가 대출 거절사유에 대한 문의시 구체적인 사유(예를 들면 연체기록 등의 신용정보가 있는지 여부 등)까지 안내토록 지도하였고, 관련 서식도 이를 반영하여 개정중에 있습니다. 따라서 대출신청 은행에 문의하면 본인의 대출이 거절된 원인도 구체적으로 알 수 있을 것입니다.\r\n\r\n            '}
{'제목': '보험계약 후 청약서와 보험약관을 교부 받지 못했는데 지금까지 낸 보험료를 모두 돌려받을 수 있나요?', '작성자': '안상현', '등록일': '2014-09-22 10:46:14', '조회수': '43561', '담당부서': '분쟁조정국', '담당팀': '제3보험팀', '문의': '3145-5747', '첨부파일': 'NULL', '답변': '\r\n\r\n\r\n\r\n                보험계약을 체결하면서 보험계약자가 청약서에 자필서명을 하지 않았거나, 약관과 청약서를 받지 못하였거나 약관의 중요한 내용에 관하여 설명을 듣지 못했을 때에는 청약일로부터 3개월 이내에 계약을

{'제목': '저축은행에 예금을 하면 예금보호를 받을 수 있나요?', '작성자': '박성진', '등록일': '2014-05-19 03:10:46', '조회수': '9874', '담당부서': '저축은행감독국', '담당팀': '관리지도팀', '문의': '02-3145-6781', '첨부파일': 'NULL', '답변': '\r\n\r\n\r\n\r\n                ㅁ\xa0상호저축은행이 취급하는 보통예금, 저축예금, 정기예금, 정기적금, 신용부금, 표지어음 등은 예금자보호법상 예금자보호대상에 해당됩니다.\n\r\n\xa0o\xa0상호저축은행의 인ㆍ허가 취소, 해산, 파산의 경우 예금자의 청구에 의하여 예금보험공사가 보험금(예금대지급)을 지급하며,\r\n\xa0o 보호한도는 1개 상호저축은행 1인당 원금과 소정의 이자를 합하여 최고 5천만원이고,\r\n\xa0o 소정의 이자는 약정이자와 예금보험공사가 결정한 이자(예금보험공사가 시중은행 1년만기 정기예금의 평균금리를 감안하여 정한 이자) 중 적은 금액을 적용합니다.\n\r\nㅁ 단, 저축은행 발행채권(후순위채권) 등은 예금자보호대상에서 제외됩니다.\r\n\r\n            '}
{'제목': '최근 부실저축은행 구조조정 과정에서 영업정지조치를 부과하지 않는 이유', '작성자': '박성진', '등록일': '2014-04-29 01:56:55', '조회수': '9572', '담당부서': '저축은행감독국', '담당팀': '관리지도팀', '문의': '02-3145-6781', '첨부파일': 'NULL', '답변': '\n\n【질문내용】\r\n\t\xa0\r\n\t최근 부실저축은행 구조조정 과정에서 영업정지조치를 부과하지 않는 이유\n\n\n【답변내용】\n\r\n\t예금자 등 저축은행 거래자들은 적기시정조치 부과 등 구조조정 과정 진행과 관계없이 지속적으로 저축은행에서 금융거래가 가능하고, 영업정지에 따른 거래 불편을 우려한 원리금 합산 5천만원 이하 예금자 등이 부실 관련 풍문 등에 따라 급격히 예금을 

{'제목': '투자권유대행인은 어떤 사람인가요?', '작성자': '김정규', '등록일': '2013-03-25 10:36:38', '조회수': '11193', '담당부서': '금융투자검사국', '담당팀': '검사기획팀', '문의': '02-3145-7017', '첨부파일': 'NULL', '답변': '\n\n투자권유대행인은 투자자의 편의성을 제고하기 위해 증권회사 임직원은 아니면서 증권회사로부터 업무위탁을 받아 투자자에게 금융투자상품의 매매의 체결 등을 권유하는 업무를 하는 사람입니다.\n\xa0 \n1개 증권회사에 전속되어 관리·감독을 받으며, 종전 투자상담사 등이 자본시장법 시행과 함께 투자권유대행인으로 통합되었습니다.\n'}
8 페이지 완료
{'제목': '지인이 증권 위탁계좌를 관리해 주겠다고 비밀번호 및 거래인감 등을 자신에게 맡기라고 하는데 괜찮나요?', '작성자': '김정규', '등록일': '2013-03-25 10:34:53', '조회수': '11590', '담당부서': '금융투자검사국', '담당팀': '검사기획팀', '문의': '02-3145-7017', '첨부파일': 'NULL', '답변': '\n\n계좌번호, 비밀번호, 거래인감과 증권카드 등을 지인에게 알리거나 보관하는 경우 횡령, 배임 등 금융사고로 이어질 개연성이 높으므로 어떠한 경우에도 지인에게 알리거나 보관토록 하지 말고 스스로 관리하고, 투자금액은 본인이 직접 계좌에 입금하여야 합니다.\n'}
{'제목': '금융투자상품을 매매거래할 때 금융투자회사와 투자자 사이에 손실부담 약정을 맺는 것이 가능한가요?', '작성자': '김정규', '등록일': '2013-03-25 10:33:23', '조회수': '12731', '담당부서': '금융투자검사국', '담당팀': '검사기획팀', '문의': '02-3145-7017', '첨부파일': 'NULL', '답변': '\n\n자본시장법에서는 금융투자업자가 증권의 매매, 그밖에 거래와 관련하여 손실을 보전하거나 이익을 보장하는 행위를 금지(법 제71조

{'제목': '채권추심이 금지되는 야간은 언제를 의미하나요?', '작성자': '이정미', '등록일': '2012-10-25 05:34:05', '조회수': '15826', '담당부서': 'IT감독실', '담당팀': '신용정보1팀', '문의': '02-3145-7429', '첨부파일': 'NULL', '답변': '\n\r\n\xa0오후 9시부터 다음날 오전 8시까지는 채무자의 평온을 보호하기 위해 채권추심이 금지되어 있습니다.\n\r\n야간에 채권을 추심하려는 경우, 핸드폰 등을 사용하여 사진 촬영, 음성녹음 등의 방법으로 증거를 확보하여\n\r\n수사기관에 신고하시기 바랍니다.\r\n\r\n            '}
{'제목': '선이자 등을 공제하는 경우 이자율 계산은 어떻게 하나요?', '작성자': '김진형', '등록일': '2012-10-25 05:29:48', '조회수': '12865', '담당부서': '서민금융지원국', '담당팀': '서민금융지원팀', '문의': '02-3145-8159', '첨부파일': 'NULL', '답변': '\r\n\r\n\r\n\r\n                \xa0대출금에서 선이자를 사전에 공제한 경우, 선이자를 제외한 실제 교부된 액수를 원금으로 하여 이자율을 계산합니다.\n\r\n즉, 계약상 100만원을 빌린것으로 되어 있으나 선이자를 10만원 공제한 경우, 실제 원금은 90만원으로 계산하는 것입니다.\n\r\n단, 담보권 설정비용 및 신용조회 비용은 공제하더라도 공제하기 전 금액을 원금으로 봅니다.\n\r\n상세한 사항은 국번없이 1332로 전화하셔서 상담 받으시기 바랍니다.\r\n\r\n            '}
{'제목': '개인회생자에 대한 금융지원제도는 없나요?', '작성자': '임동빈', '등록일': '2012-10-25 05:26:12', '조회수': '14611', '담당부서': '서민금융지원국', '담당팀': '서민금융총괄팀', '문의': '02-3145-8124', '첨부파일': 'NULL', '답변

...

{'제목': '은행 등 금융기관에 대한 모집수수료 지급방법에는 어떤 제한이 있습니까?', '작성자': '박영조', '등록일': '2005-05-10 01:55:06', '조회수': '7572', '담당부서': '보험감독국', '담당팀': '보험업무팀', '문의': '02-3145-7462', '첨부파일': 'NULL', '답변': '\n\n\n\n\n\n\n\n\n\r\n\t\t\t\t\t\t보험업법규에서 모집수수료 지급방법을 제한하는 기준은 없습니다. 보험업감독규정 별표7-2 제2호 다목에서 모집수수료 이외에 여타의 성과급을 지급하지 아니한다는 규정도 모집수수료 지급방법을 제한하는 기준은 아닙니다..\n\n\n\n\n\n\n'}
{'제목': '금융기관보험대리점에서 채용한 보험설계사는 어떠한 경우라도 1년 이상을 채용하여야 합니까?', '작성자': '박영조', '등록일': '2005-05-10 01:54:29', '조회수': '7905', '담당부서': '보험감독국', '담당팀': '보험업무팀', '문의': '02-3145-7462', '첨부파일': 'NULL', '답변': '\n\n\n\n\n\n\n금융기관보험대리점은 감독규정 제4-14조에서 정한 기준에 따라 보험설계사를 채용한 경우 특별한 사유가 없는 한 채용기간을 1년 이상으로 하여야 합니다.\n그러나 채용된 보험설계사의 부당행위 등 금융기관보험대리점의 취업규칙상 계속 채용이 불가능한 사유가 발생할 경우에는 동 기준을 준수하지 아니 할 수 있습니다.\n\n\n\n\n\n\n'}
{'제목': '금융기관보험대리점이 채용한 보험설계사는 보험모집 이외의 다른 업무를 취급할 수 있습니까?', '작성자': '박영조', '등록일': '2005-05-10 01:53:48', '조회수': '8156', '담당부서': '보험감독국', '담당팀': '보험업무팀', '문의': '02-3145-7462', '첨부파일': 'NULL', '답변': '\n\n\n\n\n\n\n보험모집 이외의 다른 업무도 취급할 수 있습니다.\r\n

{'제목': '보험회사가 가지급보험금 지급을 지연하거나 가지급보험금을 적게 산정하여 지급하는 경우에는 어떻게 하나요?', '작성자': '김우택', '등록일': '2005-05-09 10:19:08', '조회수': '27946', '담당부서': '보험상품감독국', '담당팀': '손해보험팀', '문의': '02-3145-8244', '첨부파일': 'NULL', '답변': '\n\n\n\n\n\n보험회사는 보험사고의 조사 및 확인 절차를 거쳐 보험금을 추산하게 되므로 가지급보험금 지급에 어느 정도 기간은 소요됩니다. 또한 가지급보험금은 추정보험금을 기준으로 지급액이 정해지며, 보험회사와 피해자(보험금청구권자)간에 추정한 보험금에 차이가 있을 수 있습니다.\n가지급보험금 지급기일 및 지급액 산정과 관련해서는 위와 같은 절차나 기준이 있음을 참고하십시오.\n보험회사가 가지급보험금 지급을 지연하거나 가지급보험금을 적게 산정하여 지급한 것으로 판단되는 경우에는 해당회사에 가지급보험금 지급 지연 사유나 가지급보험금 산정기준 등에 대한 설명을 요구하고 보험회사의 업무처리가 부당하다고 생각되는 경우 해당회사의 본사 고객센터나 금융감독원 민원상담센터(국번없이 1332번)에 문의하여 도움을 받으십시오.\n\n\n\n\n\n'}
{'제목': '보험회사가 가지급보험금 지급을 거절하는 경우에는 어떻게 하나요?', '작성자': '김우택', '등록일': '2005-05-09 10:16:09', '조회수': '17459', '담당부서': '보험상품감독국', '담당팀': '손해보험팀', '문의': '02-3145-8244', '첨부파일': 'NULL', '답변': '\n\n\n\n\n\n일반적으로 보험회사는 사고에 대한 보상책임이 있는 경우 특별한 사유없이 가지급보험금 지급을 거절하지는 않을 것입니다. 그 이유는 보험회사가 피해자에게 최종적으로 확정된 보험금을 지급하는 경우 이미 지급한 가지급보험금을 공제하기 때문입니다.\n보험회사가 가지급보험금 지급을 거절하는 경우에는 해당회사에 거절하는 사

{'제목': '외국인 취득한도란 무엇인가요?', '작성자': '이승진', '등록일': '2005-05-06 07:46:25', '조회수': '7590', '담당부서': '금융투자감독국', '담당팀': '증권시장팀', '문의': '02-3145-7615', '첨부파일': 'NULL', '답변': '\n\xa0외국인 주식취득 제한 제도는 국가기간산업 등 국민경제상 중요한 산업을 영위하는 유가증권시장 및 코스닥시장 상장 법인 중 이 법령에 따라 지정되는 공공적 법인(한국전력) 또는 다른 법령에서 정하는 주식에 대해 외국인이 취득할 수 있는 한도를 두는 제도입니다\n\r\n전체 취득한도 종목은 금감원 홈페이지 -> 업무자료 -> 금융투자업무 -> 외국인국내투자동향에 있는 월중 외국인투자자 증권매매동향의\xa0\r\n마지막 페이지에 게시하고 있습니다.\n'}
{'제목': '10년 전에 주식을 사서 투자하던 국민이 이민을 가서 외국시민권자가 된 상태에서 기존에 보유한 주식을 처분하려고 할 때 투자등록증이 필요한가요?', '작성자': '한소은', '등록일': '2005-05-06 07:45:14', '조회수': '11349', '담당부서': '금융투자감독국', '담당팀': '증권시장팀', '문의': '02-3145-7609', '첨부파일': 'NULL', '답변': '\n한국국적 보유자가 외국시민권자가 되는 경우 기존에 보유한 주식을 처분하기 위해서는 외국인투자등록을 하여야 합니다. 투자등록을 하여 투자등록증을 발급받은 후 비거주자 전용계좌를 개설하여 해당 계좌로 주식을 입고하고 금융감독원에 장외양수도 신고를 하면 됩니다. \n'}
{'제목': '6개월이상 국내거주한 외국인은 투자등록증을 발급 받을 수 없나요?', '작성자': '한소은', '등록일': '2005-05-06 07:44:31', '조회수': '9139', '담당부서': '금융투자감독국', '담당팀': '증권시장팀', '문의': '02-3145-7609', '첨부파일': 'NULL', '답변': '\n\n\n\

{'제목': '기업신용위험 상시평가제도란 무엇인가요?', '작성자': '김원준', '등록일': '2005-05-06 02:59:29', '조회수': '7585', '담당부서': '기업금융개선국', '담당팀': '기업금융개선3팀', '문의': '02-3145-8373', '첨부파일': 'http://www.fss.or.kr/download.bbs?bbsid=1208250172921&fidx=1231846149134', '답변': '\n\n\n\n\n\n\n\n채권은행이 정기 또는 수시로\xa0부실화 가능성이 있다고 판단되는 기업의 신용위험을 평가하고 동 평가결과에 따라 구조조정 등 사후관리를 추진하는 제도입니다.\n\n금융기관의 신용공여 규모가 500억원 이상인 기업은 「기업구조조정촉진법」에 따라, 그리고 500억원 미만 기업은 「은행감독업무시행세칙」에 따라 채권은행이 자율적으로 구축한 상시평가시스템으로 평가됩니다.\n\n\n\n\n\n'}
{'제목': '기업 워크아웃(Workout)이란 무엇인가요?', '작성자': '김원준', '등록일': '2005-05-06 02:57:57', '조회수': '7596', '담당부서': '기업금융개선국', '담당팀': '기업금융개선3팀', '문의': '02-3145-8373', '첨부파일': 'http://www.fss.or.kr/download.bbs?bbsid=1208250172921&fidx=1231846245674', '답변': '\n워크아웃(Workout)이란 ‘재무적으로 곤경에 처했으나 경제적으로는 회생가능성이 있는 기업과 채권금융기관이 협력하여 재무구조와 사업구조를 조정하는 방식으로 기업회생과 채권회수 증대를 꾀하는 일련의 과정’으로서 기업개선작업이라고도 불립니다.\xa0\n\n\n‘98.6월 외환위기시 기업회생을 위해 210개 금융기관 간 체결된 ｢기업구조조정 촉진을 위한 금융기관 협약｣으로 기본체계가 만들어졌고, ’01.8월 「기업구조조정촉진법」의 제정에 따라 상시기업구조조정시스템으로 정착되었습니다.\xa0\n

{'제목': '사모펀드의 특징은 무엇입니까?', '작성자': '김아진', '등록일': '2005-05-06 11:02:24', '조회수': '8536', '담당부서': '자산운용감독실', '담당팀': '사모펀드팀', '문의': '02-3145-6716', '첨부파일': 'NULL', '답변': '\n\n\n\n\n\n\n\n\r\n\t\t\t\t\t불특정 다수를 상대로 모집 매출하는 공모 펀드의 경우 투자설명서 설명 및 제공, 자산운용방법에 대한 제한, 펀드 운용에 대한 공시 등 투자자 보호를 위한 제도가 마련되어 있는 반면, 사모펀드는 공모 펀드에 비하여 투자자 보호를 위한 제도가 상당 부분 적용되지 않는 특징을 가지고 있습니다.\n\n\n\n\n\n'}
{'제목': '펀드의 종류는 어떻게 구분됩니까?', '작성자': '김명수', '등록일': '2005-05-06 11:01:47', '조회수': '8781', '담당부서': '자산운용감독실', '담당팀': '상품심사팀', '문의': '02-3145-6713', '첨부파일': 'NULL', '답변': '\n펀드의 종류는 투자자 모집의 방법에 따라 공모펀드와 사모펀드로 구분되며, 펀드의 투자대상에 따라 증권(주식형, 채권형, 혼합형),\xa0부동산, 특별자산, 혼합자산, 단기금융(MMF)\xa0펀드로 구분됩니다.\n'}
{'제목': '환매연기란 무엇입니까?', '작성자': '김명수', '등록일': '2005-05-06 11:00:35', '조회수': '8849', '담당부서': '자산운용감독실', '담당팀': '상품심사팀', '문의': '02-3145-6713', '첨부파일': 'NULL', '답변': '\n자산운용회사 등은 투자자의 환매청구가 있는 경우 펀드의 자산을 매각하여 조성한 금전으로만 환매에 응해야 합니다.\n펀드의 자산이 매각이 불가능하여 사실상 환매에 응할 수 없는 경우나, 환매에 응하는 것이 투자자의 이익 또는 형평성을 해할 가능성이 있는 경우 등의 사유가 있는 때에는 자산운용회사 등은 환매연기를

In [ ]:
def main():
    total_qnaList = list()
    fileName = '5_금융감독원(금융소비자보호처)'
    csv_file = open(str(fileName) + '.csv', "w", encoding = 'utf-8-sig', newline = '')
    
    columns = ['제목', '작성자', '등록일', '조회수', '담당부서', '담당팀', '문의', '첨부파일','답변']
    
    wr = csv.writer(csv_file)    
    wr.writerow(columns)
    
    for idx in tqdm_notebook(range(1, 94)):
        url = 'http://consumer.fss.or.kr/fss/consumer/flguide/beware/faq/list.jsp?url=/fss/cm/1208250172921&bbsid=1208250172921&page=%s'%(str(idx))
        qnaList = get_qnaList(url)
        
        for qna in qnaList:
            qnaContent = get_qnaContent(qna)
            print(qnaContent)
            total_qnaList.append(qnaContent)
        print(idx, '페이지 완료')
    
    
    # '모든 수집 복지서비스'에 대한 반복문 -> 복지 서비스 하나
    for info_dict in total_qnaList:
        # 'columns'을 돌면서 info_dict가 해당 col을 가지고 있으면 정보를 적고, 없으면 NULL
        wr.writerow([info_dict.get(col, 'NULL') for col in columns])
    
    print('==================== 저장 완료 =======================')
    csv_file.close()
             
    